In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from lewidi_lib import enable_logging, load_preds

enable_logging()

folder = "../parquets/perspectivist/"
rdf = load_preds(folder)

In [ ]:
from lewidi_lib import process_rdf


rdf = process_rdf(rdf, task="perspectivist", discard_invalid_pred=True)

In [ ]:
from lewidi_lib import assign_cols_perf_metrics_softlabel, join_correct_responses


joint_df = join_correct_responses(rdf, task="perspectivist")

In [ ]:
row = joint_df.query("dataset == 'VariErrNLI'").iloc[10]
print(row["reasoning"])

In [ ]:

joint_df.query("dataset == 'VariErrNLI' and n_annotations != n_annotators")